In [1]:
import sys
from pathlib import Path

project_root = Path("../")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from typing import List, TypedDict, Literal

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

from src.utils import load_vectorstore, load_llm
from src.storage import Storage


/Users/user/Documents/study/learning_langchain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_path = "../data/db"
storage_path = "../data/processed"

print("벡터 저장소 로드 중...")
vectorstore = load_vectorstore(db_path)
print("벡터 저장소 로드 완료")

print("\nStorage 초기화 중...")
storage = Storage(storage_path)
print("Storage 초기화 완료")

print("\nLLM 초기화 중...")
llm = load_llm()
if llm:
    print("LLM 초기화 완료")
else:
    print("LLM 초기화 실패 (GPT_API_KEY 확인 필요)")


벡터 저장소 로드 중...


/Users/user/Documents/study/learning_langchain/weekly_project/../src/utils.py:30: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


벡터 저장소 로드 완료

Storage 초기화 중...
Storage 초기화 완료

LLM 초기화 중...
LLM 초기화 완료


In [3]:
class ChatState(TypedDict):
    messages: List[BaseMessage]
    intent: str
    query: str
    context: str

In [4]:
def classify_intent(state: ChatState) -> ChatState:
    query = state.get("query", "")
    messages = state.get("messages", [])
    
    messages.append(HumanMessage(content=query))
    state["messages"] = messages
    
    if not llm:
        state["intent"] = "SMALL_TALK"
        return state
    
    prompt = f"""다음 사용자 발화를 보고, 의도를 아래 중 하나로 골라라: DOC_QA, SUMMARY, SMALL_TALK. 다른 말은 하지 말고, 태그만 출력해라.

사용자 발화: {query}

의도:"""
    
    try:
        response = llm.invoke(prompt)
        intent = response.content.strip() if hasattr(response, 'content') else str(response).strip()
        
        if intent not in ["DOC_QA", "SUMMARY", "SMALL_TALK"]:
            if any(keyword in query.lower() for keyword in ["요약", "정리", "핵심", "요점"]):
                intent = "SUMMARY"
            elif any(keyword in query.lower() for keyword in ["안녕", "뭐야", "누구", "소개"]):
                intent = "SMALL_TALK"
            else:
                intent = "DOC_QA"
        
        state["intent"] = intent
    except Exception as e:
        print(f"의도 분류 오류: {e}")
        state["intent"] = "SMALL_TALK"
    
    return state


In [5]:
def retrieve_documents(query: str, k: int = 5) -> List[Document]:
    return vectorstore.similarity_search(query, k=k)

def load_original_content(doc: Document) -> str:
    doc_type = doc.metadata.get('type', 'text')
    
    if doc_type == 'table':
        table_id = doc.metadata.get('table_id')
        if table_id:
            table_data = storage.get_table(table_id)
            if table_data:
                markdown = table_data.get('markdown', '')
                description = table_data.get('description', '')
                return f"{description}\n{markdown}" if description else markdown
    else:
        chunk_id = doc.metadata.get('chunk_id')
        text_id = doc.metadata.get('text_id') or f'text_{chunk_id}' if chunk_id is not None else None
        
        if text_id:
            text_data = storage.get_text(text_id)
            if text_data:
                return text_data.get('original_text', doc.page_content)
    
    return doc.page_content

def build_context(docs: List[Document]) -> str:
    context_parts = []
    for i, doc in enumerate(docs, 1):
        original_content = load_original_content(doc)
        doc_type = doc.metadata.get('type', 'text')
        
        if doc_type == 'table':
            table_id = doc.metadata.get('table_id', '')
            context_parts.append(f"[테이블 {i}: {table_id}]\n{original_content}")
        else:
            context_parts.append(f"[텍스트 {i}]\n{original_content}")
    
    return "\n\n".join(context_parts)


In [6]:
def doc_qa_node(state: ChatState) -> ChatState:
    query = state.get("query", "")
    messages = state.get("messages", [])
    
    docs = retrieve_documents(query, k=5)
    context = build_context(docs)
    state["context"] = context
    
    if not llm:
        answer = "LLM이 설정되지 않아 답변을 생성할 수 없습니다."
    else:
        chat_history = "\n".join([f"{'사용자' if isinstance(msg, HumanMessage) else '어시스턴트'}: {msg.content}" for msg in messages[:-1]])
        
        prompt = f"""다음 문서들을 참고하여 질문에 답변해주세요.

참고 문서:
{context}

대화 기록:
{chat_history}

질문: {query}

답변:"""
        
        try:
            response = llm.invoke(prompt)
            answer = response.content if hasattr(response, 'content') else str(response)
        except Exception as e:
            answer = f"답변 생성 중 오류 발생: {e}"
    
    messages.append(AIMessage(content=answer))
    state["messages"] = messages
    
    return state


In [7]:
def summary_node(state: ChatState) -> ChatState:
    query = state.get("query", "")
    messages = state.get("messages", [])
    context = state.get("context", "")
    
    if not context:
        docs = retrieve_documents(query, k=5)
        context = build_context(docs)
        state["context"] = context
    
    if not llm:
        answer = "LLM이 설정되지 않아 요약을 생성할 수 없습니다."
    else:
        chat_history = "\n".join([f"{'사용자' if isinstance(msg, HumanMessage) else '어시스턴트'}: {msg.content}" for msg in messages[:-1]])
        
        prompt = f"""다음 문서 내용을 요약해주세요. 사용자의 요청에 맞게 요약해주세요.

문서 내용:
{context}

대화 기록:
{chat_history}

사용자 요청: {query}

요약:"""
        
        try:
            response = llm.invoke(prompt)
            answer = response.content if hasattr(response, 'content') else str(response)
        except Exception as e:
            answer = f"요약 생성 중 오류 발생: {e}"
    
    messages.append(AIMessage(content=answer))
    state["messages"] = messages
    
    return state


In [8]:
def small_talk_node(state: ChatState) -> ChatState:
    query = state.get("query", "")
    messages = state.get("messages", [])
    
    if not llm:
        answer = "안녕하세요! 저는 기술 문서를 기반으로 질문에 답변하는 AI 어시스턴트입니다."
    else:
        chat_history = "\n".join([f"{'사용자' if isinstance(msg, HumanMessage) else '어시스턴트'}: {msg.content}" for msg in messages[-5:]])
        
        prompt = f"""당신은 친절한 AI 어시스턴트입니다. 사용자와 자연스럽게 대화하세요.

대화 기록:
{chat_history}

사용자: {query}
어시스턴트:"""
        
        try:
            response = llm.invoke(prompt)
            answer = response.content if hasattr(response, 'content') else str(response)
        except Exception as e:
            answer = f"답변 생성 중 오류 발생: {e}"
    
    messages.append(AIMessage(content=answer))
    state["messages"] = messages
    
    return state


In [9]:
def router_edge_fn(state: ChatState) -> Literal["doc_qa", "summary", "small_talk"]:
    intent = state.get("intent", "SMALL_TALK")
    
    if intent == "DOC_QA":
        return "doc_qa"
    elif intent == "SUMMARY":
        return "summary"
    else:
        return "small_talk"


In [10]:
def build_graph():
    workflow = StateGraph(ChatState)
    
    workflow.add_node("router", classify_intent)
    workflow.add_node("doc_qa", doc_qa_node)
    workflow.add_node("summary", summary_node)
    workflow.add_node("small_talk", small_talk_node)
    
    workflow.set_entry_point("router")
    
    workflow.add_conditional_edges(
        "router",
        router_edge_fn,
        {
            "doc_qa": "doc_qa",
            "summary": "summary",
            "small_talk": "small_talk"
        }
    )
    
    workflow.add_edge("doc_qa", END)
    workflow.add_edge("summary", END)
    workflow.add_edge("small_talk", END)
    
    memory = MemorySaver()
    return workflow.compile(checkpointer=memory)

app = build_graph()
print("그래프 구성 완료")


그래프 구성 완료


In [11]:
# LangGraph 내장 시각화 방법
print("=== ASCII 그래프 ===")
app.get_graph().print_ascii()

print("\n=== Mermaid 다이어그램 ===")
print(app.get_graph().draw_mermaid())



=== ASCII 그래프 ===
                     +-----------+                       
                     | __start__ |                       
                     +-----------+                       
                           *                             
                           *                             
                           *                             
                      +--------+                         
                      | router |.                        
                   ...+--------+ ....                    
               ....        .         ....                
           ....            .             ....            
         ..                .                 ..          
+--------+          +------------+          +---------+  
| doc_qa |*         | small_talk |          | summary |  
+--------+ ****     +------------+       ***+---------+  
               ****        *         ****                
                   ****    *     ****                 

In [12]:
thread_id = "test-user-1"

while True:
    q = input("\n질문: ").strip()
    
    if not q:
        continue
    
    if q.lower() in ["exit", "quit"]:
        print("종료합니다.")
        break
    
    config = {"configurable": {"thread_id": thread_id}}
    
    result = app.invoke(
        {"query": q},
        config=config
    )
    
    answer = result["messages"][-1].content
    intent = result.get('intent', 'UNKNOWN')
    
    print(f"\n[질문] {q}")
    print(f"[의도: {intent}]")
    
    if intent == "DOC_QA" and result.get("context"):
        print(f"\n[참고 문서]\n{result['context']}")
        print(f"\n{'='*60}")
    
    print(f"\n[답변]\n{answer}")



[의도: SMALL_TALK]

[답변]
안녕하세요! 어떻게 도와드릴까요?

[의도: SUMMARY]

[답변]
문서 요약:

이 보고서는 테슬라(Tesla, Inc.)의 2025년 3분기(9월 30일 종료)에 대한 10-Q 양식입니다. 주요 내용은 다음과 같습니다:

1. **재무 정보** (Part I):
   - **재무제표**:
     - 대차대조표, 운영 성과, 종합 소득, 비지배 지분 및 자본 관련 제무제표, 현금 흐름표 등이 포함됨.
   - **경영진의 논의 및 분석** (Item 2): 재무 상태 및 운영 성과에 대한 논의.
   - **시장 위험에 대한 정량적 및 정성적 공시** (Item 3).
   - **통제 및 절차** (Item 4).

2. **기타 정보** (Part II):
   - 법적 절차, 위험 요소, 주식 미등록 판매, 채무 불이행, 광산 안전 관련 정보, 기타 정보 및 전시물 항목이 포함됨.

첨부된 재무 데이터는 모두 감사되지 않은 자료로, 이전 연도의 재무제표와 비교하여 작성되었습니다. 이에 따라 매출, 서비스 및 기타 수익원에 대한 세부사항이 제공됩니다.

보고서에는 각 항목의 페이지 번호와 함께 상세한 내용이 나열되어 있습니다.

[의도: DOC_QA]

[참고 문서]
[테이블 1: table_39]
Tesla’s Form 10‑Q (Q3 2025) discloses revenue by geography for the three months ended September 30 2025 and the nine months ended September 30 2025, showing total quarterly revenue of **$28.1 billion** (U.S. **$14.6 B**, China **$5.7 B**, other international **$7.8 B**) and nine‑month revenue of **$71.98 billion** (U.S. **$36.74 B**, China **$1